In [ ]:
#!/usr/bin/env python3
"""Search for PlanetScope scenes over Milan region in January 2025."""

import json
from planetscope_py import PlanetScopeQuery
from planetscope_py.utils import calculate_area_km2

def create_milan_polygon():
    """Create a 100km x 100km polygon centered on Milan, Italy."""
    
    # Milan city center coordinates
    milan_lat = 45.4642  # Milan latitude
    milan_lon = 9.1900   # Milan longitude
    
    print(f"📍 Milan center: {milan_lat}°N, {milan_lon}°E")
    
    # To create a 100km x 100km box, we need to calculate the lat/lon offsets
    # At Milan's latitude (~45°N):
    # - 1 degree latitude ≈ 111 km
    # - 1 degree longitude ≈ 111 * cos(45°) ≈ 78.5 km
    
    # Calculate offsets for 50km in each direction (total 100km box)
    lat_offset = 50 / 111.0  # ≈ 0.45 degrees
    lon_offset = 50 / (111.0 * 0.707)  # ≈ 0.64 degrees (cos(45°) ≈ 0.707)
    
    # Create bounding box coordinates
    north = milan_lat + lat_offset
    south = milan_lat - lat_offset
    east = milan_lon + lon_offset
    west = milan_lon - lon_offset
    
    print(f"📏 Bounding box:")
    print(f"   North: {north:.4f}°")
    print(f"   South: {south:.4f}°")
    print(f"   East:  {east:.4f}°")
    print(f"   West:  {west:.4f}°")
    
    # Create polygon geometry (clockwise from northwest)
    milan_polygon = {
        "type": "Polygon",
        "coordinates": [[
            [west, north],   # Northwest corner
            [east, north],   # Northeast corner
            [east, south],   # Southeast corner
            [west, south],   # Southwest corner
            [west, north]    # Close the polygon
        ]]
    }
    
    # Calculate actual area
    area_km2 = calculate_area_km2(milan_polygon)
    print(f"📐 Actual polygon area: {area_km2:.2f} km²")
    
    return milan_polygon

def search_milan_january_2025():
    """Search for PlanetScope scenes over Milan in January 2025."""
    
    print("🚀 SEARCHING FOR PLANETSCOPE SCENES OVER MILAN")
    print("=" * 60)
    
    # Create Milan polygon
    milan_geometry = create_milan_polygon()
    
    # Initialize query
    query = PlanetScopeQuery()
    
    print(f"\n🔍 Searching for scenes in January 2025...")
    print(f"   Date range: 2025-01-01 to 2025-01-31")
    print(f"   Area: Milan region (100km x 100km)")
    print(f"   Cloud cover: ≤ 20%")
    
    try:
        # Search for scenes
        results = query.search_scenes(
            geometry=milan_geometry,
            start_date="2025-01-01",
            end_date="2025-01-31",
            cloud_cover_max=0.2,  # 20% cloud cover
            item_types=["PSScene"]
        )
        
        # Process results
        scenes = results.get('features', [])
        num_scenes = len(scenes)
        
        print(f"\n📊 SEARCH RESULTS:")
        print(f"   Total scenes found: {num_scenes}")
        
        if num_scenes == 0:
            print("   ℹ️ No scenes found for these criteria.")
            print("   💡 Try expanding the date range or increasing cloud cover threshold.")
            return results
        
        # Analyze scenes by date
        scenes_by_date = {}
        cloud_covers = []
        
        for scene in scenes:
            props = scene.get('properties', {})
            acquired = props.get('acquired', 'unknown')
            cloud_cover = props.get('cloud_cover', 0)
            
            # Extract date
            date = acquired.split('T')[0] if 'T' in acquired else acquired
            
            if date not in scenes_by_date:
                scenes_by_date[date] = []
            scenes_by_date[date].append(scene)
            cloud_covers.append(cloud_cover)
        
        # Summary statistics
        avg_cloud_cover = sum(cloud_covers) / len(cloud_covers) if cloud_covers else 0
        min_cloud_cover = min(cloud_covers) if cloud_covers else 0
        max_cloud_cover = max(cloud_covers) if cloud_covers else 0
        
        print(f"\n📈 SCENE STATISTICS:")
        print(f"   Days with imagery: {len(scenes_by_date)}")
        print(f"   Average cloud cover: {avg_cloud_cover:.1%}")
        print(f"   Cloud cover range: {min_cloud_cover:.1%} - {max_cloud_cover:.1%}")
        
        # Show daily breakdown
        print(f"\n📅 DAILY BREAKDOWN:")
        for date in sorted(scenes_by_date.keys()):
            daily_scenes = scenes_by_date[date]
            daily_clouds = [s.get('properties', {}).get('cloud_cover', 0) for s in daily_scenes]
            avg_daily_cloud = sum(daily_clouds) / len(daily_clouds) if daily_clouds else 0
            
            print(f"   {date}: {len(daily_scenes)} scenes (avg cloud: {avg_daily_cloud:.1%})")
        
        # Show some example scenes
        print(f"\n📸 EXAMPLE SCENES (first 5):")
        for i, scene in enumerate(scenes[:5]):
            props = scene.get('properties', {})
            scene_id = scene.get('id', 'unknown')
            acquired = props.get('acquired', 'unknown')
            cloud_cover = props.get('cloud_cover', 0)
            
            print(f"   {i+1}. {scene_id}")
            print(f"      Acquired: {acquired}")
            print(f"      Cloud cover: {cloud_cover:.1%}")
            print()
        
        return results
        
    except Exception as e:
        print(f"❌ Search error: {e}")
        import traceback
        traceback.print_exc()
        return None

def save_results_to_file(results, filename="milan_scenes_jan2025.json"):
    """Save search results to a JSON file."""
    
    if not results:
        print("❌ No results to save")
        return
    
    try:
        with open(filename, 'w') as f:
            json.dump(results, f, indent=2)
        
        print(f"💾 Results saved to: {filename}")
        print(f"   File size: {len(json.dumps(results))} characters")
        
    except Exception as e:
        print(f"❌ Error saving results: {e}")

def analyze_coverage_quality(results, milan_geometry):
    """Analyze the coverage quality of the found scenes."""
    
    if not results or not results.get('features'):
        print("❌ No scenes to analyze")
        return
    
    print(f"\n🎯 COVERAGE QUALITY ANALYSIS:")
    print("=" * 60)
    
    scenes = results['features']
    milan_area_km2 = calculate_area_km2(milan_geometry)
    
    print(f"   Target area (Milan): {milan_area_km2:.2f} km²")
    
    # Analyze scene coverage (simplified)
    total_scenes = len(scenes)
    high_quality_scenes = [s for s in scenes if s.get('properties', {}).get('cloud_cover', 1) <= 0.1]
    medium_quality_scenes = [s for s in scenes if 0.1 < s.get('properties', {}).get('cloud_cover', 1) <= 0.2]
    
    print(f"   Total scenes: {total_scenes}")
    print(f"   High quality (≤10% cloud): {len(high_quality_scenes)}")
    print(f"   Medium quality (10-20% cloud): {len(medium_quality_scenes)}")
    
    # Find best scenes
    if high_quality_scenes:
        print(f"\n🌟 BEST QUALITY SCENES (≤10% cloud):")
        for i, scene in enumerate(high_quality_scenes[:3]):
            props = scene.get('properties', {})
            scene_id = scene.get('id', 'unknown')
            acquired = props.get('acquired', 'unknown')
            cloud_cover = props.get('cloud_cover', 0)
            
            print(f"   {i+1}. {scene_id} - {acquired} (cloud: {cloud_cover:.1%})")

def main():
    """Main function to search for Milan data."""
    
    print("🗺️  MILAN REGION PLANETSCOPE DATA SEARCH")
    print("=" * 80)
    
    # Search for scenes
    results = search_milan_january_2025()
    
    if results:
        # Create the geometry for analysis
        milan_geometry = create_milan_polygon()
        
        # Analyze coverage
        analyze_coverage_quality(results, milan_geometry)
        
        # Save results
        save_results_to_file(results)
        
        print(f"\n🎉 SEARCH COMPLETED SUCCESSFULLY!")
        print(f"   Found {len(results.get('features', []))} scenes over Milan in January 2025")
        print(f"   Results saved to milan_scenes_jan2025.json")
    else:
        print(f"\n❌ Search failed or no results found")
    
    print("=" * 80)

if __name__ == "__main__":
    main()